# Imports #

In [5]:
import pandas as pd
import numpy as np
import datetime
import time
from decouple import config
import requests
import urllib.parse
import json
AAapikey = config('AAKey')

# Gather Historic Stock News #

In [6]:
#Function to convert user provided date to date required by AlphaAdvantage
def toAADate(oldDate):
    newDate = oldDate.strftime("%Y%m%dT0130")
    return str(newDate)

In [11]:
def getHistoricNewsData(ticker, endDate = datetime.date.today() - datetime.timedelta(days = 1), interval = 365):
    # Format for news is [Date, Headline, Ticker]
    historic_news = pd.DataFrame(columns=['Date', 'Headline', 'Ticker'])
    url = 'https://www.alphavantage.co/query?'
    window = 15
    startDate = endDate - datetime.timedelta(days = interval)
    windowStart = startDate
    windowEnd = startDate + datetime.timedelta(days = window)
    delta = datetime.timedelta(days = window)
    count = 0
    while (windowEnd <= endDate):
        if count == 5:
            time.sleep(60)
            count = 0
        startDateAA = toAADate(windowStart)
        endDateAA = toAADate(windowEnd)
        Myparams = {'function': 'NEWS_SENTIMENT', 'tickers': ticker, 'time_from': startDateAA, 'time_to': endDateAA, 'sort': 'EARLIEST','limit': 200, 'apikey': AAapikey}
        r = requests.get(url, params = Myparams)
        data = r.json()
        try:
            for i in data.get("feed"):
                test_date = i.get("time_published")
                test_date = test_date[:8]
                newDate = datetime.datetime.strptime(test_date, '%Y%m%d').date() 
                row = [newDate, i.get("title"), ticker]
                new_df = pd.DataFrame([row],columns=['Date', 'Headline', 'Ticker'])
                historic_news = pd.concat([historic_news, new_df], axis=0, ignore_index=True)
        except:
            print("Request failed")
            print("Starting date: " +str(windowStart))
            print("Starting date: " +str(windowEnd))
            print(data)
            break
        windowStart = windowEnd
        windowEnd = windowEnd + delta
        count = count + 1
    print("Final rolling date is: " +str(windowEnd))
    #print(historic_parsed_news)
    return historic_news


In [13]:
hist = getHistoricNewsData('AAPL')

Final rolling date is: 2023-02-25


In [15]:
print(hist.head)

<bound method NDFrame.head of             Date                                           Headline Ticker
0     2022-03-01  US stocks fall, oil tops $105 as Ukraine crisi...   AAPL
1     2022-03-02  Apple halts product sales in Russia after Ukra...   AAPL
2     2022-03-02  Russia says its economy is taking 'serious blo...   AAPL
3     2022-03-02  Rich Russians turn to luxury jewellery, watche...   AAPL
4     2022-03-03  Market Rallies, But Still Must Do This; 5 Stoc...   AAPL
...          ...                                                ...    ...
4596  2023-02-01  23 Things That Didn't Exist When Tom Brady Ent...   AAPL
4597  2023-02-01  Meta stock spikes despite earnings miss, as Fa...   AAPL
4598  2023-02-01                 Morning Bid: Riding the Fed dragon   AAPL
4599  2023-02-01  Meta Revenue Beats As Company Announces $40 Bi...   AAPL
4600  2023-02-01  Fed Chief Triggers 'Gratifying' Rally; Meta Sp...   AAPL

[4601 rows x 3 columns]>


In [16]:
hist.to_csv('AAPL-365-day-hist.csv')